In [ ]:
from dataclasses import dataclass
@dataclass
class Config:
    rows: int
    columns: int
    inarow: int  
@dataclass
class Params:
    weight_num_threes_opp: float
    weight_num_fours: float

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)

env.run([agent, LookOneAheadAgent(Params(weight_num_threes_opp=10, weight_num_fours=100))])

env.render(mode="ipython")

In [ ]:
import numpy as np
import random

def score_move(grid, col, piece, config, params):
    ngrid = drop_piece(grid, col, piece, config)
    score = get_heuristic(ngrid, piece, config, params)
    return score

def drop_piece(grid, col, piece, config):
    ngrid = grid.copy()
    
    for row in range(config.rows - 1, -1, -1):
        if grid[row][col] == 0:
            break
    ngrid[row][col] = piece
    return ngrid

def get_heuristic(grid, piece, config, params):
    num_threes = count_window(grid, 3, piece, config)
    num_threes_opp = count_window(grid, 3, piece % 2 + 1, config)
    num_fours = count_window(grid, 4, piece, config)
    
    score = num_threes - params.weight_num_threes_opp * num_threes_opp + params.weight_num_fours * num_fours
    return score

def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
def count_window(grid, num_discs, piece, config):
    num_windows = 0
    
    # horizontal
    for r in range(config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r][c1] for c1 in range(c, c + config.inarow)]
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for c in range(config.columns):
        for r in range(config.rows - (config.inarow - 1)):
            window = [grid[r1][c] for r1 in range(r, r + config.inarow)]
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # diagonal
    for r in range(config.rows - (config.inarow - 1)):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r + i][c + i] for i in range(config.inarow)]
            #print(window)
            if check_window(window, num_discs, piece, config):
                num_windows += 1
            window = [grid[r + config.inarow - 1 - i][c + i] for i in range(config.inarow)]
            #print(window)
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

In [ ]:
def agent(obs, config):
    
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, Params(100, 1e6)) for col in valid_moves]))
    
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    
    return random.choice(max_cols)

In [ ]:
class LookOneAheadAgent:
    def __init__(self, params):
        self._params = params
    
    def __call__(self, obs, config):
        
        #Get list of valid moves
        valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]

        # Convert the board to a 2D grid
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)

        # Use the heuristic to assign a score to each possible board in the next turn
        scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, self._params) for col in valid_moves]))

        # Get a list of columns (moves) that maximize the heuristic
        max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

        # Select at random from the maximizing columns
        return random.choice(max_cols)

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
get_win_percentages(agent1=LookOneAheadAgent(Params(weight_num_threes_opp=10, weight_num_fours=100)),
                    agent2="random")

In [ ]:
weights3 = [4, 8, 10, 20, 40]
weights4 = [20, 50, 100, 200, 400]

In [ ]:
for w3 in  weights3:
    for w4 in weights4:
        print(w3, w4)
        print(get_win_percentages(agent1=LookOneAheadAgent(Params(weight_num_threes_opp=w3, weight_num_fours=w4)),
                                  agent2=LookOneAheadAgent(Params(weight_num_threes_opp=10, weight_num_fours=100))))